In [3]:
import pandas as pd

class Node:
    def __init__(self, level, profit, weight, bound, items):
        self.level = level
        self.profit = profit
        self.weight = weight
        self.bound = bound
        self.items = items

def bound(node, n, max_days, profits, weights):
    if node.weight >= max_days:
        return 0
    
    profit_bound = node.profit
    j = node.level + 1
    total_weight = node.weight

    while j < n and total_weight + weights[j] <= max_days:
        total_weight += weights[j]
        profit_bound += profits[j]
        j += 1

    if j < n:
        profit_bound += (max_days - total_weight) * profits[j] / weights[j]

    return profit_bound

def branch_and_bound(n, profits, weights, max_days):
    items = list(range(n))
    items.sort(key=lambda i: profits[i] / weights[i], reverse=True)
    
    sorted_profits = [profits[i] for i in items]
    sorted_weights = [weights[i] for i in items]
    
    queue = []
    u = Node(-1, 0, 0, 0.0, [])
    v = Node(0, 0, 0, 0.0, [])
    
    max_profit = 0
    queue.append(u)

    while queue:
        u = queue.pop(0)
        
        if u.level == -1:
            v.level = 0
        if u.level == n - 1:
            continue
        
        v.level = u.level + 1
        
        v.weight = u.weight + sorted_weights[v.level]
        v.profit = u.profit + sorted_profits[v.level]
        v.items = u.items + [items[v.level]]
        
        if v.weight <= max_days and v.profit > max_profit:
            max_profit = v.profit
            best_items = v.items
        
        v.bound = bound(v, n, max_days, sorted_profits, sorted_weights)
        
        if v.bound > max_profit:
            queue.append(v)
        
        v = Node(v.level, u.profit, u.weight, 0.0, u.items[:])
        v.bound = bound(v, n, max_days, sorted_profits, sorted_weights)
        
        if v.bound > max_profit:
            queue.append(v)
    
    return max_profit, best_items

# Read the CSV file
df = pd.read_csv('projects.csv')

# Check the structure of the DataFrame
print(df)

# Extract the relevant data
projects = df.columns[1:].astype(int)  # Assuming the first column is 'Project' and the rest are project indices
revenues = df.iloc[0, 1:].astype(int).tolist()  # Assuming the first row contains revenues
days = df.iloc[1, 1:].astype(int).tolist()  # Assuming the second row contains days

# Define the total available researcher days
total_days = 150  # Example value; adjust as needed

n = len(projects)
max_profit, selected_projects = branch_and_bound(n, revenues, days, total_days)

print("Maximum Profit:", max_profit)
print("Selected Projects:", [projects[i] for i in selected_projects])


   Project   1   2   3   4   5   6
0  Revenue  15  20   5  25  22  17
1     Days  51  60  40  62  63  10
Maximum Profit: 64
Selected Projects: [6, 4, 5]
